In [3]:
# %matplotlib inline
# import mpld3
# mpld3.enable_notebook()
import rosbag_pandas
import pandas as pd
import matplotlib.pyplot as plt
import easygui
import rosbag
import json_tricks as json
import numpy as np
import cPickle as pkl
import time
import seaborn as sns
from matplotlib.patches import Rectangle, Circle
# import mpld3
# mpld3.enable_notebook()
def TicTocGenerator():
    # Generator that returns time differences
    ti = 0           # initial time
    tf = time.time() # final time
    while True:
        ti = tf
        tf = time.time()
        yield tf-ti # returns the time difference

TicToc = TicTocGenerator() # create an instance of the TicTocGen generator

# This will be the main function through which we define both tic() and toc()
def toc(tempBool=True):
    # Prints the time difference yielded by generator instance TicToc
    tempTimeInterval = next(TicToc)
    if tempBool:
        print( "Elapsed time: %f seconds.\n" %tempTimeInterval )

def tic():
    # Records a time in TicToc, marks the beginning of a time interval
    toc(False)
    
def pickler( obj, path):
    """
    Pickle a Python object
    """
    with open(path, "wb") as pfile:
        pkl.dump(obj, pfile)

def depickler( path):
    """
    Extracts a pickled Python object and returns it
    """
    with open(path, "rb") as pfile:
        data = pkl.load(pfile)
    return data
def circles(x, y, s, c='b',ax=None, vmin=None, vmax=None, **kwargs):
    """
    Make a scatter of circles plot of x vs y, where x and y are sequence 
    like objects of the same lengths. The size of circles are in data scale.

    Parameters
    ----------
    x,y : scalar or array_like, shape (n, )
        Input data
    s : scalar or array_like, shape (n, ) 
        Radius of circle in data unit.
    c : color or sequence of color, optional, default : 'b'
        `c` can be a single color format string, or a sequence of color
        specifications of length `N`, or a sequence of `N` numbers to be
        mapped to colors using the `cmap` and `norm` specified via kwargs.
        Note that `c` should not be a single numeric RGB or RGBA sequence 
        because that is indistinguishable from an array of values
        to be colormapped. (If you insist, use `color` instead.)  
        `c` can be a 2-D array in which the rows are RGB or RGBA, however. 
    vmin, vmax : scalar, optional, default: None
        `vmin` and `vmax` are used in conjunction with `norm` to normalize
        luminance data.  If either are `None`, the min and max of the
        color array is used.
    kwargs : `~matplotlib.collections.Collection` properties
        Eg. alpha, edgecolor(ec), facecolor(fc), linewidth(lw), linestyle(ls), 
        norm, cmap, transform, etc.

    Returns
    -------
    paths : `~matplotlib.collections.PathCollection`

    Examples
    --------
    a = np.arange(11)
    circles(a, a, a*0.2, c=a, alpha=0.5, edgecolor='none')
    plt.colorbar()

    License
    --------
    This code is under [The BSD 3-Clause License]
    (http://opensource.org/licenses/BSD-3-Clause)
    """
    import numpy as np
    import matplotlib.pyplot as plt
    from matplotlib.patches import Circle
    from matplotlib.collections import PatchCollection

    if np.isscalar(c):
        kwargs.setdefault('color', c)
        c = None
    if 'fc' in kwargs: kwargs.setdefault('facecolor', kwargs.pop('fc'))
    if 'ec' in kwargs: kwargs.setdefault('edgecolor', kwargs.pop('ec'))
    if 'ls' in kwargs: kwargs.setdefault('linestyle', kwargs.pop('ls'))
    if 'lw' in kwargs: kwargs.setdefault('linewidth', kwargs.pop('lw'))

    patches = [Circle((x_, y_), s_) for x_, y_, s_ in np.broadcast(x, y, s)]
    collection = PatchCollection(patches, **kwargs)
    if c is not None:
        collection.set_array(np.asarray(c))
        collection.set_clim(vmin, vmax)
    if ax is not None:
        ax = plt.gca()
    ax.add_collection(collection)
    ax.autoscale_view()
    if c is not None:
        plt.sci(collection)
    return collection

In [28]:
np.random.seed(0)
np.random.rand(3,2)

array([[ 0.5488135 ,  0.71518937],
       [ 0.60276338,  0.54488318],
       [ 0.4236548 ,  0.64589411]])

In [77]:
import matplotlib as mpl
# df=metadata=None
def quadPlotter(df,metadata,pth='plot.pdf',col=[0,0,0,1],off=20,
                xmin=506,xmax=520,ymin=508,ymax=522,rm=10,s=4,xt='X (m)',yt='Y (m)'):
    
    parameters=metadata['parameters']
        # row and column sharing
    fig, ((ax2, ax1), (ax3, ax4)) = plt.subplots(2, 2, sharex=True, sharey=True,figsize=(14,14))
    ax={4:ax4,1:ax1,2:ax2,3:ax3}
    maxS=df.trajectory__speed.max()
    # ax1.set_title('lr anal')
    for i in range(int(df.trajectory__case.max()+1)):
        df2=df[(df.trajectory__case==i) & (df.trajectory__runNum<20) &(df.trajectory__isFlying==True)]
        c1=[1,0,0,0.5]
        c2=[0,1,0,0.5]

#         circles(df2.trajectory__o1Pos_x[0],df2.trajectory__o1Pos_y[0],ax=ax[i+1],color=c1,lw=0,s=parameters['obj1Scale']/2)
#         circles(df2.trajectory__o2Pos_x[0],df2.trajectory__o2Pos_y[0],ax=ax[i+1],color=c2,lw=0,s=parameters['obj1Scale']/2)
        sc=0.7
        r1=sc*parameters['obj1Scale']/2
        r2=sc*parameters['obj2Scale']/2
        
        ls=parameters['loadingString']
        non=[0,0,0,0]
        red=[1,0,0,0.3]
        green=[0,1,0,0.3]
        wf=3 #wifth fat
        w0=0 #width thin
        
        if ls=='00':
            l=non
            r=non
            w1=wf
            w2=wf
        elif ls=='01':
            l=non
            r=red
            w1=wf
            w2=w0
        elif ls=='10':
            l=green
            r=non
            w1=w0
            w2=wf
        elif ls=='11':
            l=green
            r=red
            w1=w0
            w2=w0

        if df2.trajectory__case[0] == 0:
            c1=l
            c2=l
            l1=w1
            l2=w1
        elif df2.trajectory__case[0]== 1:
            c1=r
            c2=l
            l1=w2
            l2=w1
        elif df2.trajectory__case[0]== 2:
            c1=l
            c2=r
            l1=w1
            l2=w2
        elif df2.trajectory__case[0] == 3:
            c1=r
            c2=r
            l1=w2
            l2=w2
        print 'c1 c2 is',c1,c2
        
#         ax[i+1].add_artist(Circle(xy=(parameters['posL'][0],parameters['posL'][1]), color=c1, radius=r1)) 

#         ax[i+1].add_artist(Circle(xy=(parameters['posR'][0],parameters['posR'][1]), color=c2, radius=r2)) 
            
        
        if df.trajectory__speedControl[0]:
            col=df2.trajectory__speed.values/maxS
        else:
            col=col
            
        im=ax[i+1].scatter(df2.trajectory__pPos_x[::rm],df2.trajectory__pPos_y[::rm],lw=0,s=s
                        ,c=col,cmap='viridis',rasterized=True)
        
        ax[i+1].scatter(parameters['playerInitPos'][0],parameters['playerInitPos'][1],c=np.array([62,218,0,200])/255.,
                        marker='^',lw=0,s=500)

        ax[i+1].scatter(parameters['posL'][0],parameters['posL'][1],c=c1,lw=l1,s=sc*parameters['obj1Scale'])
        ax[i+1].scatter(parameters['posR'][0],parameters['posR'][1],c=c2,lw=l2,s=sc*parameters[ 'obj2Scale'])

        ax[i+1].set_xlim([xmin,xmax])
        ax[i+1].set_ylim([ymin,ymax])

        ax[i+1].set_aspect('equal', 'datalim')
#         ax[i+1].set_title(i)
        
        ax[i+1].set_xlabel(xt)
        ax[i+1].set_ylabel(yt)

        print i
    
#     # Colorbar
#     ax3.cax.colorbar(ax3)
#     ax3.cax.toggle_label(True)
#     fig.colorbar(im, ax=ax3)
    
#     cax,kw = mpl.colorbar.make_axes([ax1,ax2,ax3,ax4])
#     plt.colorbar(im, cax=cax, **kw)
        plt.tight_layout()
#         fig.text(0.5, 0.04, xt, ha='center')
#         fig.text(0.04, 0.5, yt, va='center', rotation='vertical')

#     sns.despine()
#     sns.set_style("ticks")
#         sns.set_style("white")
#         sns.set_style("whitegrid")


#         sns.set_style("ticks", {"xtick.major.size": 8, "ytick.major.size": 8})
#     fig.subplots_adjust(right=0.8)
    
    if df.trajectory__speedControl[0]:

        cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
        fig.colorbar(im, cax=cbar_ax)
    
    plt.savefig(pth, dpi=400)

    plt.show()
    return fig

def loader():
    '''Load a dataframe from pickle and analyse lr cases'''

    defaultPath="/home/rhagoletis/catkin/src/World/bags/"
    path=easygui.fileopenbox(title="Choose pickle Dataframes"
                              ,default=defaultPath,
                              multiple=False,filetypes=["*.pickle"])
    print path

    dff=pd.read_pickle(path)
    df=dff['df']
    metadata=dff['metadata']
    return df,metadata,path


In [80]:
df,metadata,path=loader()
parameters=metadata['parameters']
pth=path.split('/')[-1].split('.bag')[0]+'.pdf'
quadPlotter(df=df,metadata=metadata,off=3,rm=2,xt='X (m)',yt='Y (m)',s=3,pth=pth,col=[0,0,0,1])

/home/rhagoletis/catkin/src/World/bags/2017_01_04/2017-01-04__22:11:20_apple15_11_traj.bag_df.pickle


KeyboardInterrupt: (None, <function _new_Index at 0x7f79d9938668>, (<class 'pandas.indexes.base.Index'>, {'data': array(['trajectory__DCoffset', 'trajectory__case',
       'trajectory__compensation', 'trajectory__gain',
       'trajectory__headingControl', 'trajectory__impose',
       'trajectory__imposeHeading', 'trajectory__imposeResponse',
       'trajectory__imposeResponseHeading',
       'trajectory__imposeResponseSmooth', 'trajectory__isFlying',
       'trajectory__o1Pos_x', 'trajectory__o1Pos_y', 'trajectory__o1Pos_z',
       'trajectory__o2Pos_x', 'trajectory__o2Pos_y', 'trajectory__o2Pos_z',
       'trajectory__pOri_x', 'trajectory__pOri_y', 'trajectory__pOri_z',
       'trajectory__pPos_x', 'trajectory__pPos_y', 'trajectory__pPos_z',
       'trajectory__packetDuration', 'trajectory__packetFrequency',
       'trajectory__quadrant', 'trajectory__reset', 'trajectory__runNum',
       'trajectory__servoAngle', 'trajectory__speed',
       'trajectory__speedControl', 'trajectory__tr

In [70]:
import matplotlib as mpl
# df=metadata=None
def quadDistPlotter(pth='plot.pdf',col=[0,0,0,1],off=20,
                xmin=506,xmax=520,ymin=508,ymax=522,rm=10,s=4,xt='X (m)',yt='Y (m)'):
    
        # row and column sharing
    fig, ((ax2, ax1), (ax3, ax4)) = plt.subplots(2, 2, sharex=True, sharey=True,figsize=(14,14))
    ax={4:ax4,1:ax1,2:ax2,3:ax3}
#     maxS=df.trajectory__speed.max()
    # ax1.set_title('lr anal')
    dfD,metadataD,paths=multLoader()
    for i in range(int(4)):
#         df,metadata,path=loader()
        df=dfD[i]
        metadata=metadataD[i]
        parameters=metadata['parameters']
        df2=df[(df.trajectory__case==2) &(df.trajectory__isFlying==True)]
        c1=[1,0,0,0.5]
        c2=[0,1,0,0.5]

#         circles(df2.trajectory__o1Pos_x[0],df2.trajectory__o1Pos_y[0],ax=ax[i+1],color=c1,lw=0,s=parameters['obj1Scale']/2)
#         circles(df2.trajectory__o2Pos_x[0],df2.trajectory__o2Pos_y[0],ax=ax[i+1],color=c2,lw=0,s=parameters['obj1Scale']/2)
        sc=0.7
        r1=sc*parameters['obj1Scale']/2
        r2=sc*parameters['obj2Scale']/2
        
        ls=parameters['loadingString']
        non=[0,0,0,0]
        red=[1,0,0,0.3]
        green=[0,1,0,0.3]
        wf=3 #wifth fat
        w0=0 #width thin
        
        if ls=='00':
            l=non
            r=non
            w1=wf
            w2=wf
        elif ls=='01':
            l=non
            r=red
            w1=wf
            w2=w0
        elif ls=='10':
            l=green
            r=non
            w1=w0
            w2=wf
        elif ls=='11':
            l=green
            r=red
            w1=w0
            w2=w0

        if df2.trajectory__case[0] == 0:
            c1=l
            c2=l
            l1=w1
            l2=w1
        elif df2.trajectory__case[0]== 1:
            c1=r
            c2=l
            l1=w2
            l2=w1
        elif df2.trajectory__case[0]== 2:
            c1=l
            c2=r
            l1=w1
            l2=w2
        elif df2.trajectory__case[0] == 3:
            c1=r
            c2=r
            l1=w2
            l2=w2
        print 'c1 c2 is',c1,c2
        
#         ax[i+1].add_artist(Circle(xy=(parameters['posL'][0],parameters['posL'][1]), color=c1, radius=r1)) 

#         ax[i+1].add_artist(Circle(xy=(parameters['posR'][0],parameters['posR'][1]), color=c2, radius=r2)) 
            
        
        if df.trajectory__speedControl[0]:
            col=df2.trajectory__speed.values/maxS
        else:
            col=col
            
        im=ax[i+1].scatter(df2.trajectory__pPos_x[::rm],df2.trajectory__pPos_y[::rm],lw=0,s=s
                        ,c=col,cmap='viridis',rasterized=True)
        
        parameters['playerInitPos']=(df2.trajectory__pPos_x[0],df2.trajectory__pPos_y[0])
        print "init pos",parameters['playerInitPos'][0]
        ax[i+1].scatter(parameters['playerInitPos'][0],parameters['playerInitPos'][1],c=np.array([62,218,0,200])/255.,
                        marker='^',lw=0,s=500)

        ax[i+1].scatter(parameters['posL'][0],parameters['posL'][1],c=c1,lw=l1,s=sc*parameters['obj1Scale'])
        ax[i+1].scatter(parameters['posR'][0],parameters['posR'][1],c=c2,lw=l2,s=sc*parameters[ 'obj2Scale'])

        ax[i+1].set_xlim([xmin,xmax])
        ax[i+1].set_ylim([ymin,ymax])

        ax[i+1].set_aspect('equal', 'datalim')
#         ax[i+1].set_title(i)
        
        ax[i+1].set_xlabel(xt)
        ax[i+1].set_ylabel(yt)

        print i
    
#     # Colorbar
#     ax3.cax.colorbar(ax3)
#     ax3.cax.toggle_label(True)
#     fig.colorbar(im, ax=ax3)
    
#     cax,kw = mpl.colorbar.make_axes([ax1,ax2,ax3,ax4])
#     plt.colorbar(im, cax=cax, **kw)
        plt.tight_layout()
#         fig.text(0.5, 0.04, xt, ha='center')
#         fig.text(0.04, 0.5, yt, va='center', rotation='vertical')

#     sns.despine()
#     sns.set_style("ticks")
#         sns.set_style("white")
#         sns.set_style("whitegrid")


#         sns.set_style("ticks", {"xtick.major.size": 8, "ytick.major.size": 8})
#     fig.subplots_adjust(right=0.8)
    
    if df.trajectory__speedControl[0]:

        cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
        fig.colorbar(im, cax=cbar_ax)
    
    plt.savefig(pth, dpi=400)

    plt.show()
    return fig

def multLoader():
    '''Load a dataframe from pickle and analyse lr cases'''

    defaultPath="/home/rhagoletis/catkin/src/World/bags/"
#     paths=easygui.fileopenbox(title="Choose pickle Dataframes"
#                               ,default=defaultPath,
#                               multiple=True,filetypes=["*.pickle"])
    paths=['/home/rhagoletis/catkin/src/World/bags/2017_01_02/2017-01-02__15:54:36_apple11_01_traj.bag_df.pickle', '/home/rhagoletis/catkin/src/World/bags/2017_01_02/2017-01-02__17:01:04_apple11_01_traj.bag_df.pickle', '/home/rhagoletis/catkin/src/World/bags/2017_01_02/2017-01-02__17:18:07_apple11_01_traj.bag_df.pickle', '/home/rhagoletis/catkin/src/World/bags/2017_01_02/2017-01-02__18:07:00_apple11_01_traj.bag_df.pickle']
    print paths
    
    df={}
    metadata={}
    
    i=0
    for path in paths:
        dff=pd.read_pickle(path)
        df[i]=dff['df']
        metadata[i]=dff['metadata']
        i+=1
    return df,metadata,paths


In [74]:
# df,metadata,path=loader()
# parameters=metadata['parameters']
# pth=path.split('/')[-1].split('.bag')[0]+'.pdf'
quadDistPlotter(off=3,rm=2,xt='X (m)',yt='Y (m)',s=3,col=[0,0,0,1],ymin=490,ymax=520,xmin=498,xmax=528)

['/home/rhagoletis/catkin/src/World/bags/2017_01_02/2017-01-02__15:54:36_apple11_01_traj.bag_df.pickle', '/home/rhagoletis/catkin/src/World/bags/2017_01_02/2017-01-02__17:01:04_apple11_01_traj.bag_df.pickle', '/home/rhagoletis/catkin/src/World/bags/2017_01_02/2017-01-02__17:18:07_apple11_01_traj.bag_df.pickle', '/home/rhagoletis/catkin/src/World/bags/2017_01_02/2017-01-02__18:07:00_apple11_01_traj.bag_df.pickle']
c1 c2 is [0, 1, 0, 0.3] [0, 0, 0, 0]
init pos 513.0
0
c1 c2 is [0, 1, 0, 0.3] [0, 0, 0, 0]
init pos 513.0
1
c1 c2 is [0, 1, 0, 0.3] [0, 0, 0, 0]
init pos 513.0
2
c1 c2 is [0, 1, 0, 0.3] [0, 0, 0, 0]
init pos 513.0
3


In [79]:
plt.show()

In [38]:
np.array([1,2,3])/2.

array([ 0.5,  1. ,  1.5])

In [ ]:
df.trajectory__case.plot()
plt.show()

In [ ]:
import matplotlib.pyplot as plt, mpld3
plt.plot([3,1,4,1,5], 'ks-', mec='w', mew=5, ms=20)
mpld3.show()

In [ ]:
# row and column sharing
f, ((ax2, ax1), (ax3, ax4)) = plt.subplots(2, 2, sharex=True, sharey=True,figsize=(16,16))
ax={4:ax4,1:ax1,2:ax2,3:ax3}
# ax1.set_title('lr anal')
for i in range(int(df.trajectory__case.max()+1)):
    df2=df[(df.trajectory__case==i) & (df.trajectory__runNum<20) &(df.trajectory__isFlying==True)]
    ax[i+1].scatter(df2.trajectory__pPos_x,df2.trajectory__pPos_y,lw=0,alpha=1,s=0.2,c=(df2.trajectory__speed,df2.trajectory__speed,1))
    ax[i+1].set_aspect('equal', 'datalim')
    ax[i+1].set_title(i)
    print i
plt.tight_layout()
plt.show()



In [ ]:
'''Load a dataframe from pickle and analyse lr cases'''

defaultPath="/home/rhagoletis/catkin/src/World/bags/"
path=easygui.fileopenbox(title="Choose pickle Dataframes"
                          ,default=defaultPath,
                          multiple=False,filetypes=["*.pickle"])
print path

dff=pd.read_pickle(path)
df=dff['df']
dff

In [ ]:
print df2.trajectory__o1Pos_x[0],df2.trajectory__o1Pos_y[0]
print df2.trajectory__o2Pos_x[0],df2.trajectory__o2Pos_y[0]

In [ ]:
df2.trajectory__o1Pos_x.plot()
df2.trajectory__o2Pos_x.plot()
df2.trajectory__o1Pos_y.plot()
df2.trajectory__o2Pos_y.plot()
plt.show()

In [ ]:
df2=df[df.trajectory__case==3]
# df2

In [ ]:
plt.scatter(df2.trajectory__pPos_x,df2.trajectory__pPos_y,lw=0)
plt.show()

In [ ]:
range(3)

In [ ]:
df.trajectory__case.max()

In [ ]:
jsonPath=paths[0].split('.bag')[0]+'.json'
jsonPath

In [ ]:
with open(jsonPath,'r') as js:
    a= js.read()

In [ ]:
json.loads(a)